In [62]:
from IPython import display
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
import omegaconf
from gymnasium import spaces
import gymnasium as gym


import mbrl.models as models
from mbrl.models import Ensemble
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util
from pyoperon.sklearn import SymbolicRegressor
import warnings

warnings.filterwarnings("ignore")


%load_ext autoreload
%autoreload 2

mpl.rcParams.update({"font.size": 16})

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
def term_fn(a, next_obs):
    #threshold = 10
    #not_done = (next_obs > -threshold)* (next_obs < threshold)
    #done = ~not_done
    #done = done[:, None]
    return False

def reward_fn(a, next_obs):
    return torch.cos(2 * torch.pi * next_obs) * torch.exp(torch.abs(next_obs) / 3)

In [86]:
class Simple1DMDP(gym.Env):
    def __init__(self):
        super(Simple1DMDP, self).__init__()
        
        # define the action space
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        
        # define the observation space: continuous single dimension for position
        self.observation_space = spaces.Box(low=-10, high=10, shape=(1,), dtype=np.float32)
        # Initialize state and episode length
        self.state = 0.0
        self.episode_length = 10
        self.current_step = 0

    def reset(self, seed=None):
        # reset the state to 0 and the step counter
        self.state = 0.0
        self.current_step = 0
        return torch.FloatTensor([self.state]), {}

    def step(self, action):  
        # update state based on action
        self.state += action
        # clip in the case we go outside of [-10,10]
        # set truncated parameter
        truncated = np.abs(self.state) > 10
        self.state = np.clip(self.state, -10.,10.)
        
        # calculate reward
        #reward = torch.cos(2 * torch.pi * self.state) * torch.exp(torch.abs(self.state) / 3)
        reward = reward_fn(torch.from_numpy(action), torch.from_numpy(self.state))
        # increment step counter
        self.current_step += 1
        # check if episode is terminated
        terminated = self.current_step >= self.episode_length
        
        # set placeholder for info
        info = {}
        return torch.FloatTensor([self.state]).flatten(), reward, terminated, truncated, info

    def render(self, mode='human'):
        # simple print rendering
        print(f"Step: {self.current_step}, State: {self.state}")

In [148]:
class SymbolicModel(Ensemble):
    def __init__(self, symbols, population_size, generations, max_length, max_depth, in_size, out_size,
                num_members, device, propagation_method, deterministic):
        super().__init__(num_members, device, propagation_method, deterministic)
        self.in_size = in_size
        self.out_size = out_size
        self.reg = SymbolicRegressor(population_size = population_size,
                                     allowed_symbols=symbols,
                                     optimizer_iterations=10,
                                     generations = generations,
                                     n_threads=32,
                                     max_length = max_length,
                                     max_depth=max_depth)
    
    def update_mean_std(self, mean_X, std_X, mean_y, std_y):
        self.mean_X = mean_X
        self.std_X = std_X
        self.mean_y = mean_y
        self.std_y = std_y
    
    def forward(self, x, rng, propagation_indices):
        next_obs= x[:,0] + x[:,1]
        reward = self.std_y*self.reg.predict(((x - self.mean_X)/self.std_X)) + self.mean_y
        preds = np.vstack((next_obs, reward)).T
        return torch.from_numpy(preds), None
    
    def loss(self, model_in, target):
        return self.reg.score(model_in, target)
    
    def eval_score(self, model_in, target):
        return self.reg.score(model_in, target)
    
    def sample_propagation_indices(
        self, batch_size: int, _rng: torch.Generator) -> torch.Tensor:
        model_len = num_members
        if batch_size % model_len != 0:
            raise ValueError(
                "To use SymbolicModel's ensemble propagation, the batch size must "
                "be a multiple of the number of models in the ensemble."
            )
        # rng causes segmentation fault, see https://github.com/pytorch/pytorch/issues/44714
        return torch.randperm(batch_size, device=self.device)

In [133]:
# Register the custom environment
gym.envs.registration.register(
    id='Simple1DMDP-v0',
    entry_point=Simple1DMDP
)


env = gym.make('Simple1DMDP-v0')

In [134]:
seed = 0
env.reset()
rng = np.random.default_rng(seed=seed)
generator = torch.Generator(device=device)
generator.manual_seed(seed)
obs_shape = env.observation_space.shape
act_shape = env.action_space.shape

In [135]:
# Neural Network

trial_length = 500
num_trials = 10
ensemble_size = 7

# Everything with "???" indicates an option with a missing value.
# Our utility functions will fill in these details using the 
# environment information
cfg_dict = {
    # dynamics model configuration
    "dynamics_model": {
        "_target_": "mbrl.models.GaussianMLP",
        "device": device,
        "num_layers": 4,
        "ensemble_size": ensemble_size,
        "hid_size": 200,
        "in_size": "???",
        "out_size": "???",
        "deterministic": True,
        "propagation_method": "fixed_model",
        # can also configure activation function for GaussianMLP
        "activation_fn_cfg": {
            "_target_": "torch.nn.SiLU"}
    },
    # options for training the dynamics model
    "algorithm": {
        "learned_rewards": True,
        "target_is_delta": True,
        "normalize": True,
    },
    # these are experiment specific options
    "overrides": {
        "trial_length": trial_length,
        "num_steps": num_trials * trial_length,
        "model_batch_size": 256,
        "validation_ratio": 0.05
    }
}
cfg = omegaconf.OmegaConf.create(cfg_dict)

In [136]:
# Symbolic Regression

trial_length = 500
num_trials = 20
ensemble_size = 1

device = "cpu"

# Everything with "???" indicates an option with a missing value.
# Our utility functions will fill in these details using the 
# environment information
cfg_dict = {
    # dynamics model configuration
    "dynamics_model": {
        "_target_": "__main__.SymbolicModel",
        "symbols": "add,sub,mul,div,constant,variable,sin,exp,abs",
        "population_size": 5000,
        "generations": 10000,
        "max_length": 30,
        "max_depth": 10,
        "in_size": "???",
        "out_size": "???",
        "device": device,
        "deterministic": True,
        "propagation_method": None,
        "num_members": ensemble_size
    },
    # options for training the dynamics model
    "algorithm": {
        "learned_rewards": True,
        "target_is_delta": True,
        "normalize": True,
    },
    # these are experiment specific options
    "overrides": {
        "trial_length": trial_length,
        "num_steps": num_trials * trial_length,
        "model_batch_size": 1,
        "validation_ratio": 0.05
    }
}
cfg = omegaconf.OmegaConf.create(cfg_dict)

In [137]:
# Create a 1-D dynamics model for this environment
dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape)
dynamics_model.set_elite([0])

# Create a gym-like environment to encapsulate the model
model_env = models.ModelEnv(env, dynamics_model, term_fn, reward_fn, generator=generator)

In [138]:
replay_buffer = common_util.create_replay_buffer(cfg, obs_shape, act_shape, rng=rng)

In [139]:
common_util.rollout_agent_trajectories(
    env,
    trial_length, # initial exploration steps
    planning.RandomAgent(env),
    {}, # keyword arguments to pass to agent.act()
    replay_buffer=replay_buffer,
    trial_length=trial_length
)

print("# samples stored", replay_buffer.num_stored)

# samples stored 500


In [140]:
agent_cfg = omegaconf.OmegaConf.create({
    # this class evaluates many trajectories and picks the best one
    "_target_": "mbrl.planning.TrajectoryOptimizerAgent",
    "planning_horizon": 3,
    "replan_freq": 1,
    "verbose": False,
    "action_lb": "???",
    "action_ub": "???",
    # this is the optimizer to generate and choose a trajectory
    "optimizer_cfg": {
        "_target_": "mbrl.planning.CEMOptimizer",
        "num_iterations": 10,
        "elite_ratio": 0.1,
        "population_size": 999,
        "alpha": 0.1,
        "device": device,
        "lower_bound": "???",
        "upper_bound": "???",
        "return_mean_elites": True,
        "clipped_normal": True
    }
})

agent = planning.create_trajectory_optim_agent_for_model(
    model_env,
    agent_cfg,
    num_particles=20
)

In [141]:
train_losses = []
val_scores = []

def train_callback(_model, _total_calls, _epoch, tr_loss, val_score, _best_val):
    train_losses.append(tr_loss)
    val_scores.append(val_score.mean().item())

In [142]:
class SymbolicModelTrainer:
    def __init__(self, dynamics_model: SymbolicModel):
        self.dynamics_model = dynamics_model
    
    def train(self, X_train, y_train, X_val, y_val):
        #X_train = np.hstack((dataset_train.transitions.obs, dataset_train.transitions.act))
        #X_val = np.hstack((dataset_val.transitions.obs, dataset_val.transitions.act))
        #y_train = dataset_train.transitions.rewards
        #y_val = dataset_val.transitions.rewards
        # reshuffling
        p_train = np.random.permutation(len(X_train))
        reg = self.dynamics_model.model.reg
        reg.fit(X_train[p_train], y_train[p_train])
        print(reg.get_model_string(reg.model_))
        return reg.score(X_train, y_train), reg.score(X_val, y_val)

In [143]:
# Create a trainer for the model
# model_trainer = models.ModelTrainer(dynamics_model, optim_lr=7.5e-4, weight_decay=3e-5)
model_trainer = SymbolicModelTrainer(dynamics_model)
# Main PETS loop
all_rewards = [0]
for trial in range(num_trials):
    print(trial)
    obs, _ = env.reset()    
    agent.reset()
    
    terminated = False
    total_reward = 0.0
    steps_trial = 0
    while not terminated:
        # --------------- Model Training -----------------
        if steps_trial == 0:
            dynamics_model.update_normalizer(replay_buffer.get_all())  # update normalizer stats
                        
            dataset_train, dataset_val = common_util.get_basic_buffer_iterators(
                replay_buffer,
                batch_size=cfg.overrides.model_batch_size,
                val_ratio=cfg.overrides.validation_ratio,
                ensemble_size=ensemble_size,
                shuffle_each_epoch=True,
                bootstrap_permutes=False,  # build bootstrap dataset using sampling with replacement
            )
            
            print(dataset_train.num_stored, dataset_val.num_stored)
            
            #process and standardize the data
            X_train = np.hstack((dataset_train.transitions.obs, dataset_train.transitions.act))
            X_val = np.hstack((dataset_val.transitions.obs, dataset_val.transitions.act))
            y_train = dataset_train.transitions.rewards
            y_val = dataset_val.transitions.rewards
            mean_X_train = np.mean(X_train, axis = 0)
            std_X_train = np.std(X_train, axis = 0)
            mean_y_train = np.mean(y_train, axis = 0)
            std_y_train = np.std(y_train, axis = 0)
            X_train_norm = (X_train - mean_X_train)/std_X_train
            y_train_norm = (y_train - mean_y_train)/std_y_train
            X_val_norm = (X_val - mean_X_train)/std_X_train
            y_val_norm = (y_val - mean_y_train)/std_y_train
            
            dynamics_model.model.update_mean_std(mean_X_train, std_X_train, mean_y_train, std_y_train)
            
            
            
                
            #model_trainer.train(
            #    dataset_train, 
            #    dataset_val=dataset_val, 
            #   num_epochs=2000, 
            #    patience=25, 
            #    callback=train_callback,
            #    silent=True)
            
            train_r2, val_r2 = model_trainer.train(X_train_norm, y_train_norm, X_val_norm, y_val_norm)
            print(train_r2, val_r2)
            
            

        # --- Doing env step using the agent and adding to model dataset ---
        next_obs, reward, terminated, truncated, _ = common_util.step_env_and_add_to_buffer(
            env, obs, agent, {}, replay_buffer)
                
        obs = next_obs
        total_reward += reward
        steps_trial += 1
        
        if steps_trial == trial_length:
            break
    
    all_rewards.append(total_reward)

0
475 25
((-0.937) + (1.351 * (exp(abs(((-0.091) * X1))) * abs((sin(((abs(((-4.243) * X1)) + sin((3.584 * X2))) - (((-0.148) + ((-0.387) * X1)) - (-12.319)))) - (((abs(((-6.166) * X1)) + (-6.848)) * abs((0.010 * X1))) - (exp((2.706 * X1)) / (4717.121 * X2))))))))
0.23200374841690063 -0.2148658037185669
1
485 25
(0.007 + (1.016 * ((((exp((-2.270)) / exp((1.313 * X1))) - (0.010 * X2)) - (exp(abs((sin((2.795 * X1)) + (0.224 * X1)))) * ((0.189 * X1) * sin(((-0.562) * X1))))) + (abs(((-0.505) * X1)) - ((exp((0.255 * X1)) - (0.491 * X1)) * abs(((-0.988) * X1)))))))
0.21947818994522095 -0.11211502552032471
2
494 26
((-0.001) + (1.030 * ((abs((-0.085)) - ((exp((0.425 * X1)) * (abs((sin(((-9.934) * X1)) * exp((-0.068)))) * ((-0.477) * X1))) + (0.824 * X1))) - (sin((0.687 + ((-0.906) * X1))) - (exp((-0.476)) - abs(((0.551 * X1) - ((-0.000) * X2))))))))
0.29566097259521484 -0.024814486503601074
3
504 26
((-0.056) + (1.013 * (exp((sin(sin((0.805 * X1))) - (((-12.892) * X2) + 21.145))) - (((0.531 *

In [144]:
num_data = 999
data = torch.zeros((num_data,2))
data[:,0] = torch.linspace(-10,10, num_data)

In [145]:
print(dynamics_model.model.reg.get_model_string(dynamics_model.model.reg.model_))

(0.047 + (1.003 * ((sin((((-0.690) + ((-1.011) * X2)) + ((-17.043) * X1))) / exp((0.126 * X2))) * ((exp((0.706 * X1)) - (((-7.049) * X1) + (-0.389))) * (((-0.031) * X1) * (1.660 * X2))))))


In [146]:
reward = dynamics_model.model.std_y*dynamics_model.model.reg.predict((data - dynamics_model.model.mean_X)/dynamics_model.model.std_X) + dynamics_model.model.mean_y

In [83]:
#with torch.no_grad():
#    reward = dynamics_model.model(data.to("cuda"), propagation_indices=np.arange(num_data))[0][:,1]

In [147]:
#plt.plot(data[:,0], reward.to("cpu"), label="Predicted reward")
plt.plot(data[:,0], reward, label="Predicted reward")
plt.plot(data[:,0], reward_fn(data[:,1], data[:,0]), label="True reward")
plt.legend()
plt.show()